# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/installation.md) before you start.

# Imports

In [3]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

## Object detection imports
Here are the imports from the object detection module.

In [4]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [8]:
# Model Name and Some Parameters
MODEL_NAME = "HeliTrack"

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('HeliTrack', 'label_map.pbtxt')

NUM_CLASSES = 11

## Load a (frozen) Tensorflow model into memory.

In [9]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [10]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [11]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [49]:
path = '/home/ubuntu/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/holdout/'

In [39]:
def get_immediate_subdirectories(a_dir):
    '''Taken from
    https://stackoverflow.com/questions/800197/how-to-get-all-of-the-immediate-subdirectories-in-python'''
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

In [56]:
for d in get_immediate_subdirectories(path):
    for subdir, dirs, files in os.walk(os.path.join(path, d)):
        print(subdir+"-tracked.avi")
#         for f in range(len(files)):
#             image_path = os.path.join(subdir, "{:06}.png".format(f))
#             print(image_path)

/home/ubuntu/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/holdout/Neovision2-Training-Heli-048-tracked.avi
/home/ubuntu/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/holdout/Neovision2-Training-Heli-045-tracked.avi
/home/ubuntu/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/holdout/Neovision2-Training-Heli-046-tracked.avi
/home/ubuntu/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/holdout/Neovision2-Training-Heli-044-tracked.avi
/home/ubuntu/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/holdout/Neovision2-Training-Heli-047-tracked.avi


In [ ]:
cv2.VideoWriter

In [ ]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/home/ubuntu/Documents/Metis/Main_Repo/Projects/HeliTrack/other/raw_data/holdout/Neovision2-Training-Heli-044/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [62]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:

        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        for d in get_immediate_subdirectories(path):
            for subdir, dirs, files in os.walk(os.path.join(path, d)):
                for f in range(len(files)):
                    image_path = os.path.join(subdir, "{:06}.png".format(f))
        
                    image = Image.open(image_path)
                    
                    # Grab the stats from image1 to use for the resultant video
                    height, width, layers =  np.array(image).shape
                    
                    # Create the OpenCV VideoWriter
                    video = cv2.VideoWriter(subdir+"-tracked.avi", # Filename
                        -1, # Negative 1 denotes manual codec selection. You can make this automatic by defining the "fourcc codec" with "cv2.VideoWriter_fourcc"
                        30, # 10 frames per second is chosen as a demo, 30FPS and 60FPS is more typical for a YouTube video
                        (width,height) # The width and height come from the stats of image1
                        )
                    
                    # the array based representation of the image will be used later in order to prepare the
                    # result image with boxes and labels on it.
                    image_np = load_image_into_numpy_array(image)
                    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                    image_np_expanded = np.expand_dims(image_np, axis=0)
                    # Actual detection.
                    (boxes, scores, classes, num) = sess.run(
                      [detection_boxes, detection_scores, detection_classes, num_detections],
                      feed_dict={image_tensor: image_np_expanded})
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                      image_np,
                      np.squeeze(boxes),
                      np.squeeze(classes).astype(np.int32),
                      np.squeeze(scores),
                      category_index,
                      use_normalized_coordinates=True,
                      line_thickness=8)
                    
                    video.write(cv2.cvtColor(np.array(image_np), cv2.COLOR_RGB2BGR))
                    
                video.release()

TypeError: Required argument 'code' (pos 2) not found